In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import math
import tensorflow as tf
import mediapipe as mp
import time

2023-05-30 20:52:21.653439: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 20:52:24.021460: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 20:52:24.026898: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 20:52:28.624625: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix,classification_report,accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (LSTM, Dense, Concatenate, Attention, Dropout, Softmax, Input, Flatten,Bidirectional,Activation,Permute, multiply, ConvLSTM2D, MaxPooling3D, TimeDistributed, Conv2D, MaxPooling2D)
from scipy import stats

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [3]:
mp_pose=mp.solutions.pose

mp_drawing=mp.solutions.drawing_utils

In [4]:
def mediapipe_detection(image,model):
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image,results

In [5]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(200,30,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(200,36,200), thickness=2, circle_radius=2) 
                                 )

In [6]:
video=cv2.VideoCapture(-1)
HEIGHT=int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
WIDTH=int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
FPS=int(video.get(cv2.CAP_PROP_FPS))

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while video.isOpened():
        ret,frame=video.read()
        image,results=mediapipe_detection(frame,pose)
        
        try:
            landmarks=results.pose_landmarks.landmark
        except:
            pass
        draw_landmarks(image,results)
        
        cv2.imshow('Opev CV FEED', image)
        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
    video.release()
    cv2.destroyAllWindows()

[ WARN:0@23.876] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
pose=[]
for res in results.pose_landmarks.landmark:
    test= np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [8]:
pose

[array([ 0.51433671,  0.69550455, -1.24353302,  0.99617231]),
 array([ 0.5424183 ,  0.62547517, -1.15751433,  0.99724591]),
 array([ 0.56052697,  0.62606549, -1.15800691,  0.99773973]),
 array([ 0.57469374,  0.62735349, -1.15808845,  0.99767315]),
 array([ 0.485277  ,  0.62790114, -1.14802945,  0.99638647]),
 array([ 0.46606714,  0.62933463, -1.14820027,  0.9962247 ]),
 array([ 0.44879714,  0.63207138, -1.1484611 ,  0.99561942]),
 array([ 0.59905386,  0.66234809, -0.63613588,  0.99842232]),
 array([ 0.42971343,  0.66682118, -0.60254246,  0.99669129]),
 array([ 0.5529412 ,  0.76476657, -1.05038822,  0.99342293]),
 array([ 0.48336792,  0.76225394, -1.03999758,  0.99088681]),
 array([ 0.75121307,  0.98935616, -0.48709685,  0.85343993]),
 array([ 0.29778564,  0.99208182, -0.30203286,  0.70535368]),
 array([ 0.8553192 ,  1.25065625, -1.22126448,  0.24168792]),
 array([ 0.21054681,  1.2766844 , -0.89588177,  0.07007063]),
 array([ 0.73515832,  1.12471962, -1.99917316,  0.20525122]),
 array([

In [9]:
num_landmarks=len(landmarks)
num_values=len(test)
num_input_values=num_values*num_landmarks

In [60]:
def extract_keypoints(results):
    """
    Processes and organizes the keypoints detected from the pose estimation model 
    to be used as inputs for the exercise decoder models
    
    """

    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose

In [11]:
DATA_PATH=os.path.join(os.getcwd(),'data')
print(DATA_PATH)

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
actions= np.array(['curl','press','curl'])
num_classes=len(actions)

no_sequences=100
sequence_length=FPS*1
start_folder=201

/home/adarsh/Comdes/Proj-OpenCV/data


In [12]:
# Build folder path
for action in actions:     
    for sequence in range(start_folder,no_sequences+start_folder):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))  
        except:
            pass

In [13]:
# Colors associated with each exercise (e.g., curls are denoted by blue, squats are denoted by orange, etc.)
colors = [(245,117,16), (117,245,16), (16,117,245)]

In [ ]:
cap=cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for idx, action in enumerate(actions):
        for sequence in range(start_folder,start_folder+no_sequences):
            for frame_num in range(sequence_length):
                ret,frame= cap.read()
                
                image,results=mediapipe_detection(frame,pose)
                
                try:
                    landmarks=results.pose_landmarks.landmark
                escept:
                    pass
                
                draw_landmarks(image,results)
                
                if frame_num==0:
                    cv2.putText(image,'STARTING COLLECTION', (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),4, cv2.LINE_AA)
                    cv2.putText(image,'Collecting  {} Video  NO- {}'.format(action,sequence),(10,10),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),8, cv2.LINE_AA)
                    cv2.putText(image,'Collecting  {} Video  NO- {}'.format(action,sequence),(10,10),cv2.FONT_HERSHEY_SIMPLEX, 1,colors[idx],4, cv2.LINE_AA)
                    cv2.imshow('Open CV FEED', image)
                    cv2.waitKey(500)
                else:
                    cv2.putText(image,'Collecting  {} Video  NO- {}'.format(action,sequence),(10,10),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),8, cv2.LINE_AA)
                    cv2.putText(image,'Collecting  {} Video  NO- {}'.format(action,sequence),(10,10),cv2.FONT_HERSHEY_SIMPLEX, 1,colors[idx],4, cv2.LINE_AA)
                    cv2.imshow('Open CV FEED', image)
                    
                # Extract and export landmarks
                keypoints=extract_keypoints(results)
                npy_path=os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)
                
                if cv2.waitKey(10) & 0xFF==ord('q'):
                    break
cap.release()
cv2.destroyAllWindows()

In [14]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

In [15]:
label_map={label:idx for idx,label in enumerate(actions)}

In [16]:
sequences, labels=[],[]
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH,action))).astype(int):
        window=[]
        for frame_num in range(sequence_length):
            
            dat=np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(dat)
        sequences.append(window)
        labels.append(label_map[action])

In [17]:
X=np.array(sequences)
y= to_categorical(labels).astype(int)
print(X.shape)
print(y.shape)

(300, 30, 132)
(300, 3)


In [18]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=1)
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=15/90,random_state=2)

In [32]:
# Callbacks to be used during neural network training
es_callback= EarlyStopping(monitor='val_loss',min_delta=5e-4,patience=10,verbose=0,mode='min')
lr_callback= ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5,min_lr=0.0001, verbose=0, mode='min')
chkpt_callback= ModelCheckpoint(filepath=DATA_PATH, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='min', save_freq=1)

# optimizer
opt= tf.keras.optimizers.Adam(learning_rate=0.01)

# some hyperparameters
batch_size=32
max_epochs=500

 # LSTM

In [24]:
# Set up Tensorboard logging and callbacks
NAME = f"ExerciseRecognition-LSTM-{int(time.time())}"
log_dir = os.path.join(os.getcwd(), 'logs', NAME,'')
tb_callback = TensorBoard(log_dir=log_dir)

callbacks = [tb_callback, es_callback, lr_callback, chkpt_callback]


In [25]:
lstm=Sequential()
lstm.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(sequence_length, num_input_values)))
lstm.add(LSTM(256, return_sequences=True, activation='relu'))
lstm.add(LSTM(128, return_sequences=False, activation='relu'))
lstm.add(Dense(128, activation='relu'))
lstm.add(Dense(64, activation='relu'))
lstm.add(Dense(actions.shape[0], activation='softmax'))
print(lstm.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 128)           133632    
                                                                 
 lstm_4 (LSTM)               (None, 30, 256)           394240    
                                                                 
 lstm_5 (LSTM)               (None, 128)               197120    
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 3)                 195       
                                                                 
Total params: 749,955
Trainable params: 749,955
Non-tr

In [26]:
lstm.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
lstm.fit(X_train,y_train, batch_size=batch_size, epochs=max_epochs, validation_data=(X_val, y_val), callbacks=callbacks)

Epoch 1/500
8/8 [==============================] - 5s 175ms/step - loss: 94675.2969 - categorical_accuracy: 0.3840 - val_loss: 160.9385 - val_categorical_accuracy: 0.3200 - lr: 0.0100
Epoch 2/500
8/8 [==============================] - 1s 108ms/step - loss: 4556.3091 - categorical_accuracy: 0.4640 - val_loss: 117.2032 - val_categorical_accuracy: 0.6600 - lr: 0.0100
Epoch 3/500
8/8 [==============================] - 1s 106ms/step - loss: 62.2003 - categorical_accuracy: 0.6080 - val_loss: 28.0357 - val_categorical_accuracy: 0.3200 - lr: 0.0100
Epoch 4/500
8/8 [==============================] - 1s 112ms/step - loss: 31.1029 - categorical_accuracy: 0.4800 - val_loss: 27.4530 - val_categorical_accuracy: 0.6600 - lr: 0.0100
Epoch 5/500
8/8 [==============================] - 1s 101ms/step - loss: 32.4222 - categorical_accuracy: 0.5720 - val_loss: 37.0916 - val_categorical_accuracy: 0.6600 - lr: 0.0100
Epoch 6/500
8/8 [==============================] - 1s 106ms/step - loss: 1143.2837 - categori

8/8 [==============================] - 1s 105ms/step - loss: 689.8017 - categorical_accuracy: 0.5840 - val_loss: 127.2939 - val_categorical_accuracy: 0.6400 - lr: 0.0100
Epoch 13/500
8/8 [==============================] - 1s 107ms/step - loss: 218.9548 - categorical_accuracy: 0.5520 - val_loss: 239.6570 - val_categorical_accuracy: 0.7000 - lr: 0.0100
Epoch 14/500
8/8 [==============================] - 1s 102ms/step - loss: 173.6681 - categorical_accuracy: 0.5720 - val_loss: 84.7197 - val_categorical_accuracy: 0.7400 - lr: 0.0020
Epoch 15/500
8/8 [==============================] - 1s 105ms/step - loss: 92.3523 - categorical_accuracy: 0.5800 - val_loss: 19.5526 - val_categorical_accuracy: 0.7600 - lr: 0.0020
Epoch 16/500
8/8 [==============================] - 1s 106ms/step - loss: 72.4785 - categorical_accuracy: 0.5680 - val_loss: 52.1308 - val_categorical_accuracy: 0.7600 - lr: 0.0020
Epoch 17/500
5/8 [=================>............] - ETA: 0s - loss: 49.7001 - categorical_accuracy: 0.6

8/8 [==============================] - 1s 113ms/step - loss: 109.8821 - categorical_accuracy: 0.6360 - val_loss: 195.7907 - val_categorical_accuracy: 0.7000 - lr: 0.0020
Epoch 18/500
8/8 [==============================] - 1s 106ms/step - loss: 125.3913 - categorical_accuracy: 0.5880 - val_loss: 43.7797 - val_categorical_accuracy: 0.7600 - lr: 0.0020


# LSTM+ Attention 

In [33]:
# Set up Tensorboard logging and callbacks
NAME = f"ExerciseRecognition-AttnLSTM-{int(time.time())}"
log_dir = os.path.join(os.getcwd(), 'logs', NAME,'')
tb_callback = TensorBoard(log_dir=log_dir)

callbacks = [tb_callback, es_callback, lr_callback, chkpt_callback]

In [34]:
def attention_block(inputs, time_steps):
    
    a=Permute((2,1))(inputs)
    a=Dense(time_steps, activation='softmax')(a)
    
    # Attention vector
    a_probs= Permute((2,1), name='attention_vec')(a)
    
    # Luong's multiplicative score
    output_attention_mul= multiply([inputs,a_probs], name='attention_mul')
    return output_attention_mul

In [35]:
HIDDEN_UNITS=256

# input
inputs= Input(shape=(sequence_length, num_input_values))
lstm_out= Bidirectional(LSTM(HIDDEN_UNITS, return_sequences=True))(inputs)

# Attention
attention_mul= attention_block(lstm_out, sequence_length)
attention_mul= Flatten()(attention_mul)

# Fully Connected Layer
x= Dense(2* HIDDEN_UNITS, activation= 'relu')(attention_mul)
x=Dropout(0.5)(x)

x=Dense(actions.shape[0], activation='softmax')(x)

AttnLSTM = Model(inputs=[inputs], outputs=x)
print(AttnLSTM.summary())tt


2023-05-30 00:53:50.107585: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-30 00:53:50.109492: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-30 00:53:50.111054: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 30, 132)]    0           []                               
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 30, 512)     796672      ['input_2[0][0]']                
 )                                                                                                
                                                                                                  
 permute_1 (Permute)            (None, 512, 30)      0           ['bidirectional_1[0][0]']        
                                                                                                  
 dense_9 (Dense)                (None, 512, 30)      930         ['permute_1[0][0]']        

2023-05-30 00:53:50.359577: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-30 00:53:50.361540: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-30 00:53:50.363042: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [36]:
AttnLSTM.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
AttnLSTM.fit(X_train, y_train, batch_size=batch_size, epochs=max_epochs, validation_data=(X_val, y_val), callbacks=callbacks)


Epoch 1/500


2023-05-30 00:53:50.801569: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-30 00:53:50.803477: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-30 00:53:50.804957: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

8/8 [==============================] - ETA: 0s - loss: 2.9991 - categorical_accuracy: 0.4680

2023-05-30 00:53:56.710091: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-30 00:53:56.711960: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-30 00:53:56.713363: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

8/8 [==============================] - 7s 312ms/step - loss: 2.9991 - categorical_accuracy: 0.4680 - val_loss: 0.5681 - val_categorical_accuracy: 0.6800 - lr: 0.0100
Epoch 2/500
8/8 [==============================] - 2s 202ms/step - loss: 0.5278 - categorical_accuracy: 0.7480 - val_loss: 0.2255 - val_categorical_accuracy: 0.9000 - lr: 0.0100
Epoch 3/500
8/8 [==============================] - 2s 195ms/step - loss: 0.2605 - categorical_accuracy: 0.9160 - val_loss: 0.0945 - val_categorical_accuracy: 1.0000 - lr: 0.0100
Epoch 4/500
8/8 [==============================] - 2s 202ms/step - loss: 0.1198 - categorical_accuracy: 0.9600 - val_loss: 0.0205 - val_categorical_accuracy: 1.0000 - lr: 0.0100
Epoch 5/500
8/8 [==============================] - 2s 208ms/step - loss: 0.2877 - categorical_accuracy: 0.8920 - val_loss: 0.0181 - val_categorical_accuracy: 1.0000 - lr: 0.0100
Epoch 6/500
8/8 [==============================] - 2s 197ms/step - loss: 0.1103 - categorical_accuracy: 0.9920 - val_loss:

8/8 [==============================] - 2s 199ms/step - loss: 0.0714 - categorical_accuracy: 0.9760 - val_loss: 0.0072 - val_categorical_accuracy: 1.0000 - lr: 0.0100
Epoch 8/500
8/8 [==============================] - 2s 198ms/step - loss: 0.0453 - categorical_accuracy: 0.9840 - val_loss: 0.0087 - val_categorical_accuracy: 1.0000 - lr: 0.0100
Epoch 9/500
8/8 [==============================] - 2s 198ms/step - loss: 0.0169 - categorical_accuracy: 1.0000 - val_loss: 0.0013 - val_categorical_accuracy: 1.0000 - lr: 0.0100
Epoch 10/500
8/8 [==============================] - 2s 197ms/step - loss: 0.0085 - categorical_accuracy: 1.0000 - val_loss: 3.6438e-04 - val_categorical_accuracy: 1.0000 - lr: 0.0100
Epoch 11/500
8/8 [==============================] - 2s 202ms/step - loss: 0.0035 - categorical_accuracy: 1.0000 - val_loss: 1.0806e-04 - val_categorical_accuracy: 1.0000 - lr: 0.0100
Epoch 12/500
8/8 [==============================] - 2s 198ms/step - loss: 0.0110 - categorical_accuracy: 0.9960 

8/8 [==============================] - 2s 198ms/step - loss: 0.0502 - categorical_accuracy: 0.9920 - val_loss: 4.5167e-04 - val_categorical_accuracy: 1.0000 - lr: 0.0100
Epoch 14/500
8/8 [==============================] - 2s 198ms/step - loss: 0.0325 - categorical_accuracy: 0.9920 - val_loss: 0.0036 - val_categorical_accuracy: 1.0000 - lr: 0.0100
Epoch 15/500
8/8 [==============================] - 2s 198ms/step - loss: 0.0113 - categorical_accuracy: 1.0000 - val_loss: 0.0061 - val_categorical_accuracy: 1.0000 - lr: 0.0100
Epoch 16/500
8/8 [==============================] - 2s 196ms/step - loss: 0.0129 - categorical_accuracy: 0.9960 - val_loss: 0.0049 - val_categorical_accuracy: 1.0000 - lr: 0.0100
Epoch 17/500
8/8 [==============================] - 2s 197ms/step - loss: 0.0073 - categorical_accuracy: 0.9960 - val_loss: 0.0015 - val_categorical_accuracy: 1.0000 - lr: 0.0020
Epoch 18/500
8/8 [==============================] - 2s 199ms/step - loss: 0.0034 - categorical_accuracy: 1.0000 - 

In [39]:
# Model map
models = {
    'LSTM': lstm, 
    'LSTM_Attention_128HUs': AttnLSTM, 
}

In [56]:
for model in models.values():
    res = model.predict(X_test, verbose=0)  
    print(res)

[[0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 9.9999994e-01 1.0985040e-28]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [6.7801192e-02 4.0923217e-01 5.2296656e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.0000000e+00 0.0000000e+00 9.9999994e-01]
 [0.000000

In [41]:
eval_results = {}
eval_results['confusion matrix'] = None
eval_results['accuracy'] = None
eval_results['precision'] = None
eval_results['recall'] = None
eval_results['f1 score'] = None

confusion_matrices = {}
classification_accuracies = {}   
precisions = {}
recalls = {}
f1_scores = {} 


In [42]:
for model_name, model in models.items():
    yhat=model.predict(X_test, verbose=0)
    y_true=np.argmax(y_test,axis=1).tolist()
    yhat=np.argmax(yhat, axis=1).tolist()
    
    confusion_matrices[model_name]=multilabel_confusion_matrix(ytrue,yhat)
    print(f"{model_name} confusion matrix : {confusion_matrices[model_name]}")
    
eval_results['confusion matrix']= confusion_matrices

LSTM confusion matrix: 
[[[18  0]
  [11  1]]

 [[ 1 11]
  [ 0 18]]]
LSTM_Attention_128HUs confusion matrix: 
[[[18  0]
  [ 0 12]]

 [[12  0]
  [ 0 18]]]


In [43]:
for model_name, model in models.items():
    yhat = model.predict(X_test, verbose=0)
    
    # Get list of classification predictions
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    
    # Model accuracy
    classification_accuracies[model_name] = accuracy_score(ytrue, yhat)    
    print(f"{model_name} classification accuracy = {round(classification_accuracies[model_name]*100,3)}%")

# Collect results 
eval_results['accuracy'] = classification_accuracies


LSTM classification accuracy = 63.333%
LSTM_Attention_128HUs classification accuracy = 100.0%


In [53]:
for model_name, model in models.items():
    save_dir = os.path.join(os.getcwd(), f"{model_name}.h5")
    model.save(save_dir) 

In [44]:
y_test

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0]])

In [45]:
model = AttnLSTM
model_name = 'AttnLSTM'

In [46]:
def calculate_angle(a,b,c):
    """
    Computes 3D joint angle inferred by 3 keypoints and their relative positions to one another
    
    """
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 


In [47]:
def get_coordinates(landmarks, mp_pose, side, joint):
    """
    Retrieves x and y coordinates of a particular keypoint from the pose estimation model
         
     Args:
         landmarks: processed keypoints from the pose estimation model
         mp_pose: Mediapipe pose estimation model
         side: 'left' or 'right'. Denotes the side of the body of the landmark of interest.
         joint: 'shoulder', 'elbow', 'wrist', 'hip', 'knee', or 'ankle'. Denotes which body joint is associated with the landmark of interest.
    
    """
    coord = getattr(mp_pose.PoseLandmark,side.upper()+"_"+joint.upper())
    x_coord_val = landmarks[coord.value].x
    y_coord_val = landmarks[coord.value].y
    return [x_coord_val, y_coord_val]            


In [48]:
def viz_joint_angle(image, angle, joint):
    """
    Displays the joint angle value near the joint within the image frame
    
    """
    cv2.putText(image, str(int(angle)), 
                   tuple(np.multiply(joint, [640, 480]).astype(int)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
    return


In [49]:
def count_reps(image, current_action, landmarks, mp_pose):
    """
    Counts repetitions of each exercise. Global count and stage (i.e., state) variables are updated within this function.
    
    """

    global curl_counter, press_counter, squat_counter, curl_stage, press_stage, squat_stage
    
    if current_action == 'curl':
        # Get coords
        shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        elbow = get_coordinates(landmarks, mp_pose, 'left', 'elbow')
        wrist = get_coordinates(landmarks, mp_pose, 'left', 'wrist')
        
        # calculate elbow angle
        angle = calculate_angle(shoulder, elbow, wrist)
        
        # curl counter logic
        if angle < 30:
            curl_stage = "up" 
        if angle > 140 and curl_stage =='up':
            curl_stage="down"  
            curl_counter +=1
        press_stage = None
        squat_stage = None
            
        # Viz joint angle
        viz_joint_angle(image, angle, elbow)
        
    elif current_action == 'press':
        
        # Get coords
        shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        elbow = get_coordinates(landmarks, mp_pose, 'left', 'elbow')
        wrist = get_coordinates(landmarks, mp_pose, 'left', 'wrist')

        # Calculate elbow angle
        elbow_angle = calculate_angle(shoulder, elbow, wrist)
        
        # Compute distances between joints
        shoulder2elbow_dist = abs(math.dist(shoulder,elbow))
        shoulder2wrist_dist = abs(math.dist(shoulder,wrist))
        
        # Press counter logic
        if (elbow_angle > 130) and (shoulder2elbow_dist < shoulder2wrist_dist):
            press_stage = "up"
        if (elbow_angle < 50) and (shoulder2elbow_dist > shoulder2wrist_dist) and (press_stage =='up'):
            press_stage='down'
            press_counter += 1
        curl_stage = None
        squat_stage = None
            
        # Viz joint angle
        viz_joint_angle(image, elbow_angle, elbow)
        
    elif current_action == 'squat':
        # Get coords
        # left side
        left_shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        left_hip = get_coordinates(landmarks, mp_pose, 'left', 'hip')
        left_knee = get_coordinates(landmarks, mp_pose, 'left', 'knee')
        left_ankle = get_coordinates(landmarks, mp_pose, 'left', 'ankle')
        # right side
        right_shoulder = get_coordinates(landmarks, mp_pose, 'right', 'shoulder')
        right_hip = get_coordinates(landmarks, mp_pose, 'right', 'hip')
        right_knee = get_coordinates(landmarks, mp_pose, 'right', 'knee')
        right_ankle = get_coordinates(landmarks, mp_pose, 'right', 'ankle')
        
        # Calculate knee angles
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        
        # Calculate hip angles
        left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
        
        # Squat counter logic
        thr = 165
        if (left_knee_angle < thr) and (right_knee_angle < thr) and (left_hip_angle < thr) and (right_hip_angle < thr):
            squat_stage = "down"
        if (left_knee_angle > thr) and (right_knee_angle > thr) and (left_hip_angle > thr) and (right_hip_angle > thr) and (squat_stage =='down'):
            squat_stage='up'
            squat_counter += 1
        curl_stage = None
        press_stage = None
            
        # Viz joint angles
        viz_joint_angle(image, left_knee_angle, left_knee)
        viz_joint_angle(image, left_hip_angle, left_hip)
        
    else:
        pass

In [71]:
def prob_viz(res, actions, input_frame, colors):
    """
    This function displays the model prediction probability distribution over the set of exercise classes
    as a horizontal bar graph
    
    """
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):        
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame


In [68]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
tf.get_logger().setLevel("ERROR")
tf.autograph.set_verbosity(1)

In [68]:
actions

array(['curl', 'press', 'curl'], dtype='<U5')

In [69]:
actions=['curl','press','squat']

In [72]:


# 1. New detection variables
sequence = []
predictions = []
res = []
threshold = 0.5 # minimum confidence to classify as an action/exercise
current_action = ''

# Rep counter logic variables
curl_counter = 0
press_counter = 0
squat_counter = 0
curl_stage = None
press_stage = None
squat_stage = None

# Camera object
cap = cv2.VideoCapture(0)

# Video writer object that saves a video of the real time test
fourcc = cv2.VideoWriter_fourcc('M','J','P','G') # video compression format
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # webcam video frame height
WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # webcam video frame width
FPS = int(cap.get(cv2.CAP_PROP_FPS)) # webcam video fram rate 

video_name = os.path.join(os.getcwd(),f"{model_name}_real_time_test.avi")
out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"MJPG"), FPS, (WIDTH,HEIGHT))

# Set mediapipe model 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detection
        image, results = mediapipe_detection(frame, pose)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)        
        sequence.append(keypoints)      
        sequence = sequence[-sequence_length:]
          
         
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]           
            predictions.append(np.argmax(res))
            current_action = actions[np.argmax(res)]
            confidence = np.max(res)
            
        #3. Viz logic
            # Erase current action variable if no probability is above threshold
            if confidence < threshold:
                current_action = ''

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
            # Count reps
            try:
                landmarks = results.pose_landmarks.landmark
                count_reps(
                    image, current_action, landmarks, mp_pose)
            except:
                pass

            # Display graphical information
            cv2.rectangle(image, (0,0), (640, 40), colors[np.argmax(res)], -1)
            cv2.putText(image, 'curl ' + str(curl_counter), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'press ' + str(press_counter), (240,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'squat ' + str(squat_counter), (490,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
         
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        # Write to video file
        if ret == True:
            out.write(image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()



[ WARN:0@3611.184] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
